# Xinyu's Thesis in Sympsi - Tutorial

_HyQu Group, ETH Zurich_

_A project by Anna Knörr & Uwe von Luepke, May 2021_

Sympy... what a wonderful world! Have you ever sat in front of way-too-long-Hamiltonians? Have you ever had to perform way-too-complicated-transformations? I'm sure you have. So, sympy will be your next best friend.

(Of course, understanding calculations by doing them by hand is very valuable. But at least being able to check the correctness of your result with some computational support is certainly desirable!)

Before we jump into the main part of this tutorial, i.e. how to use sympy to automate RWA approximations from Xinyu's thesis, here's a very nice introduction to the building blocks of Sympy: https://docs.sympy.org/latest/tutorial/manipulation.html.
I highly recommend it in order to get a first understanding of what a Symbol in sympy is and how it is manipulated. Knowing what "Add" or "Mul" objects are etc. will be especially useful if you want to have a look at the code behind the functions in the hyqu.py file we will be using later.

**Sympy vs. Sympsi**

Now, you might be wondering whether there is a typo in the title. Nope. Sympsi is a Python package based on sympy, but designed specifically for symbolical quantum mechanics. Documentation can be found here: https://github.com/sympsi/sympsi

This package has many useful tools for the HyQu Group, such as boson operators which we see much of later. In particular, the file _qutility.py_ proved to be a good starting point for this project since it is equipped with functions such as _hamiltonian_transformation()_. Based on this file, we wrote a new file called _hyqu.py_ with functions to automate Xinyu's thesis. At present, this work is a pilot project that will hopefully be taken further in the future.

This being said, let's get started!

In [1]:
from sympy import *
init_printing()

In case you're wondering, _init_printing()_ will simply print our outputs in a nice format.

In [53]:
from sympsi import *
from sympsi.boson import MultiBosonOp, BosonFockKet, BosonOp

As hinted at earlier, we will be using boson operators. But what's up with the MultiBosonOp? Well, let's instantiate some operators and find out why.

In [54]:
mq = MultiBosonOp("m", 0, 'discrete')
m1 = MultiBosonOp("m", 1, 'discrete')
m2 = MultiBosonOp("m", 2, 'discrete')

We won't need the following two lines for RWA, but they're just for a taste of how one can work with states and operators in sympsi :) 

In [63]:
q = BosonOp("q")
ket = BosonFockKet(0); ket

❘0⟩

In [67]:
ket._apply_operator_BosonOp(q)

0

The notation might be a bit confusing at first, so let's walk it through. We will use **mq** as our qubit operator, denoted as _q_ in Xinyu's thesis. The phonon operators will be **m1** and **m2**, for simplicity we will stick to only two modes for now. Furthermore, MultiBosonOp creates bosonic annihilation operators that belong together - in the sense that all MultiBosonOp labelled with _'m'_ will commute. This is why we chose the MultiBosonOp version over BosonOp because indeed, we want the qubit and phonon modes to naturally commute. Let's check this:

In [4]:
Commutator(mq, m1).doit()

0

So far, so good. Now, let's build the Hamiltonian from section 2.1. This means we need to define our symbols.

# Building the Hamiltonian

In [5]:
a,g1, g2, Om1, Om2, t, Hsym = symbols("alpha, g_1, g_2, Omega_1, Omega_2, t, H")
wq, wk1, wk2, wj1, wj2 = symbols("omega_q, omega_k1, omega_k2, omega_j1, omega_j2")

H0 = wq * Dagger(mq) * mq + wk1 * Dagger(m1) * m1  
H_nonlinear = -a/2 * Dagger(mq) * Dagger(mq) * mq * mq
H_coupling = g1 * Dagger(mq) * m1 + conjugate(g1) * mq * Dagger(m1)
H_drive = Om1 * Dagger(mq) * exp(-I*wj1*t) + conjugate(Om1)* mq * exp(I*wj1*t)
H = H0 + H_nonlinear + H_coupling + H_drive
Eq(Hsym, H)

                             2                                                
                         ⎛ †⎞   2                                             
        -ⅈ⋅ω_j1⋅t  †   α⋅⎝m ⎠ ⋅m        †          †          †      ⅈ⋅ω_j1⋅t 
H = Ω₁⋅ℯ         ⋅m  - ────────── + g₁⋅m ⋅m + ωₖ₁⋅m ⋅m + ω_q⋅m ⋅m + ℯ        ⋅
                           2                                                  

              
              
__     __    †
Ω₁⋅m + g₁⋅m⋅m 
              

Now, let's try out some transformations. So, let's first enter the rotating frame of the qubit using **U1** and then test this unitary on our qubit and phonon operators.

In [6]:
U1 = exp(I *t*H0); Eq(symbols("U_1"), U1)

          ⎛     †          †  ⎞
      ⅈ⋅t⋅⎝ωₖ₁⋅m ⋅m + ω_q⋅m ⋅m⎠
U₁ = ℯ                         

In [18]:
from sympsi.hyqu import unitary_transformation
mq1 = unitary_transformation(U1, mq, N=3); Eq(symbols("m_0^{'}"), mq1)

using simplified commutator


            -ⅈ⋅ω_q⋅t  
m_0__{'} = ℯ        ⋅m

In [19]:
m11 = unitary_transformation(U1, m1, N=3); Eq(symbols("m_1^{'}"), m11)

using simplified commutator


            -ⅈ⋅ωₖ₁⋅t  
m_1__{'} = ℯ        ⋅m

Looks good. In case you're wondering: The _unitary_transformation()_ function in hyqu differs slightly from the one in qutility.py. There was a little problem with commutators not being evaluated which is why we introduced a bool parameter called _simplify_commutator_ which is set to True by default. Check the file for more information.

Now, let's try to transform our entire Hamiltonian. We'll also want to simplify the resulting expression slightly by pulling together the exponentials and introducing some new symbols...

In [9]:
from sympsi.hyqu import hamiltonian_transformation
H1 = hamiltonian_transformation(U1, H, N=3).expand(); H1

using simplified commutator


                                  2                                           
                              ⎛ †⎞   2                                        
    -ⅈ⋅ω_j1⋅t  ⅈ⋅ω_q⋅t  †   α⋅⎝m ⎠ ⋅m        -ⅈ⋅ωₖ₁⋅t  ⅈ⋅ω_q⋅t  †      ⅈ⋅ω_j1⋅
Ω₁⋅ℯ         ⋅ℯ       ⋅m  - ────────── + g₁⋅ℯ        ⋅ℯ       ⋅m ⋅m + ℯ       
                                2                                             

                                             
                                             
t  -ⅈ⋅ω_q⋅t __      ⅈ⋅ωₖ₁⋅t  -ⅈ⋅ω_q⋅t __    †
 ⋅ℯ        ⋅Ω₁⋅m + ℯ       ⋅ℯ        ⋅g₁⋅m⋅m 
                                             

In [66]:
from sympsi.hyqu import simplify_exp
dk1, dj1 = symbols("delta_k1, delta_j1")
sub = [(wj1-wq,dj1), (wk1-wq,dk1)]
H1 = simplify_exp(H1, sub); Eq(symbols("H_1"), H1)

                              2                                               
                          ⎛ †⎞   2                                            
         -ⅈ⋅δ_j1⋅t  †   α⋅⎝m ⎠ ⋅m        -ⅈ⋅δₖ₁⋅t  †      ⅈ⋅δ_j1⋅t __      ⅈ⋅δ
H₁ = Ω₁⋅ℯ         ⋅m  - ────────── + g₁⋅ℯ        ⋅m ⋅m + ℯ        ⋅Ω₁⋅m + ℯ   
                            2                                                 

            
            
ₖ₁⋅t __    †
    ⋅g₁⋅m⋅m 
            

Next up would be the Schrieffer-Wolf transformation. We still encountered some difficulties but for the sake of documentation, here's how far we got with a very simple **U2** using just one parameter $\lambda_1$.

In [15]:
l1 = symbols("lambda_1")
U2 = exp(conjugate(l1)*Dagger(m1)*mq*exp(I*dk1*t) - l1*m1*Dagger(mq)*exp(-I*dk1*t)); Eq(symbols("U_2"), U2)

            -ⅈ⋅δₖ₁⋅t    †    ⅈ⋅δₖ₁⋅t __  †  
      - λ₁⋅ℯ        ⋅m⋅m  + ℯ       ⋅λ₁⋅m ⋅m
U₂ = ℯ                                      

In [20]:
mq2 = unitary_transformation(U2, mq, N=3); Eq(symbols("m_0^{''}"), mq2)

using simplified commutator


                             ⎛     __    ⎞  
                -ⅈ⋅δₖ₁⋅t     ⎜  λ₁⋅λ₁    ⎟  
m_0__{''} = λ₁⋅ℯ        ⋅m + ⎜- ───── + 1⎟⋅m
                             ⎝    2      ⎠  

In [21]:
m12 = unitary_transformation(U2, m1, N=3); Eq(symbols("m_1^{'}"), m12)

using simplified commutator


           ⎛     __    ⎞                  
           ⎜  λ₁⋅λ₁    ⎟      ⅈ⋅δₖ₁⋅t __  
m_1__{'} = ⎜- ───── + 1⎟⋅m - ℯ       ⋅λ₁⋅m
           ⎝    2      ⎠                  

Indeed, this is how we want our operators to transform up to second order in $\lambda$. The Hamiltonian wouldn't quite work out - we'll try to keep working on this.



# Higher-Order RWA 

With this in mind, we'll leave out the displacement transformation **U3** for now. Also, we'll follow Xinyu's thesis and demonstrate the implementation of higher order RWA approximations by focusing on specific terms from that way-too-long Hamiltonian on p. 6. More specifically, we'll choose to work with the terms **H10** and **H13** and package our symbols more conveniently to generate these two terms. 

In [22]:
dk1, dk2, dj1, dj2 = symbols("delta_k1, delta_k2, delta_j1, delta_j2")
l1, l2, x1, x2 = symbols("lambda_1, lambda_2, xi_1, xi_2")
dk = [dk1, dk2]
dj = [dj1, dj2]
l = [l1, l2]
x = [x1, x2]
m = [m1, m2]

H10 = 0
H13 = 0
sub = [a, wq, wk1, wk2, dk1, dk2, dj1, dj2]
for h in range(2):
    for i in range(1):
        for j in range(1):
            H10 += conjugate(x[h])*x[i]*x[j]*Dagger(mq)*exp(I*t*(-1*a*Dagger(mq)*mq + dj[h]-dj[i]-dj[j]))
for k in range(2):
    for j in range(1):
        for s in range(1): # using s instead of l for ease of notation
            H13 += x[j]*conjugate(l[k])*l[s]*Dagger(m[k])*m[s]*Dagger(mq)*exp(I*t*(-1*a*Dagger(mq)*mq + dj[k]-dk[j]-dk[s]))
            
H10 += conjugate_expression(H10, sub)
H13 += conjugate_expression(H13, sub)
H10 *= -a
H13 *= -2 * a
H10 = H10.expand()
H13 = H13.expand()

Let's briefly have a look at what terms we are now talking about:

In [26]:
Eq(symbols("H_{10}"), H10)

                                 †                                      †     
               2 __  †  - ⅈ⋅α⋅t⋅m ⋅m - ⅈ⋅δ_j1⋅t       2 __  †  - ⅈ⋅α⋅t⋅m ⋅m - 
H_{10} = - α⋅ξ₁ ⋅ξ₁⋅m ⋅ℯ                        - α⋅ξ₁ ⋅ξ₂⋅m ⋅ℯ               

                               2             †                     2          
2⋅ⅈ⋅δ_j1⋅t + ⅈ⋅δ_j2⋅t        __     ⅈ⋅α⋅t⋅m⋅m  + ⅈ⋅δ_j1⋅t        __     ⅈ⋅α⋅t⋅
                      - α⋅ξ₁⋅ξ₁ ⋅m⋅ℯ                      - α⋅ξ₂⋅ξ₁ ⋅m⋅ℯ      

   †                        
m⋅m  + 2⋅ⅈ⋅δ_j1⋅t - ⅈ⋅δ_j2⋅t
                            

In [27]:
Eq(symbols("H_{13}"), H13)

                                          †                                   
                     __  †    †  - ⅈ⋅α⋅t⋅m ⋅m + ⅈ⋅δ_j1⋅t - 2⋅ⅈ⋅δₖ₁⋅t          
H_{13} = - 2⋅α⋅λ₁⋅ξ₁⋅λ₁⋅m ⋅m⋅m ⋅ℯ                                    - 2⋅α⋅λ₁⋅

                        †                                                     
   __  †    †  - ⅈ⋅α⋅t⋅m ⋅m + ⅈ⋅δ_j2⋅t - 2⋅ⅈ⋅δₖ₁⋅t          __ __    †    ⅈ⋅α⋅
ξ₁⋅λ₂⋅m ⋅m⋅m ⋅ℯ                                    - 2⋅α⋅λ₁⋅λ₁⋅ξ₁⋅m⋅m ⋅m⋅ℯ    

     †                                                        †               
t⋅m⋅m  - ⅈ⋅δ_j1⋅t + 2⋅ⅈ⋅δₖ₁⋅t          __ __    †    ⅈ⋅α⋅t⋅m⋅m  - ⅈ⋅δ_j2⋅t + 2
                              - 2⋅α⋅λ₂⋅λ₁⋅ξ₁⋅m⋅m ⋅m⋅ℯ                         

        
⋅ⅈ⋅δₖ₁⋅t
        

Right. To obtain the RWA coefficients, we will follow the following recipe:

1) Generate the **"building blocks"**. (We've already got $H_{10}$ and $H_{13}$, but we still want their integrals $\int_t\ H_{10} dt$ and $\int_t\ H_{13} dt$. For this, we will be using _time_integral()_

2) **Multiply** building blocks together, according to the formula for the RWA order of interest, e.g. (2.5.1) in Xinyu's thesis.

3) Use the _trace()_ function to **eliminate terms** that would kill the qubit ground state. 

4) Use the _time_average()_ function to, well, perform the time averaging and thus keep only those terms that fulfill the **resonance condition**.

Let's try it!

In [28]:
from sympsi.hyqu import time_integral, trace, time_average, simplify_exp

In [31]:
H10_int = time_integral(H10); Eq(symbols("H_{10}^{int}"), H10_int)

                                        †                                     
                      2 __  †  - ⅈ⋅α⋅t⋅m ⋅m - 2⋅ⅈ⋅δ_j1⋅t + ⅈ⋅δ_j2⋅t       2 __
                  α⋅ξ₁ ⋅ξ₂⋅m ⋅ℯ                                       α⋅ξ₁ ⋅ξ₁
H_{10}__{int} = - ───────────────────────────────────────────────── - ────────
                                  -α - 2⋅δ_j1 + δ_j2                          

              †                       2             †                     2   
  †  - ⅈ⋅α⋅t⋅m ⋅m - ⅈ⋅δ_j1⋅t        __     ⅈ⋅α⋅t⋅m⋅m  + ⅈ⋅δ_j1⋅t        __    
⋅m ⋅ℯ                          α⋅ξ₁⋅ξ₁ ⋅m⋅ℯ                        α⋅ξ₂⋅ξ₁ ⋅m⋅
──────────────────────────── - ───────────────────────────────── - ───────────
     -α - δ_j1                              α + δ_j1                          

          †                        
 ⅈ⋅α⋅t⋅m⋅m  + 2⋅ⅈ⋅δ_j1⋅t - ⅈ⋅δ_j2⋅t
ℯ                                  
───────────────────────────────────
   α + 2⋅δ_j1 - δ_j2               

In [33]:
H13_int = time_integral(H13); Eq(symbols("H_{13}^{int}"), H13_int)

                                                 †                            
                            __  †    †  - ⅈ⋅α⋅t⋅m ⋅m + ⅈ⋅δ_j2⋅t - 2⋅ⅈ⋅δₖ₁⋅t   
                  2⋅α⋅λ₁⋅ξ₁⋅λ₂⋅m ⋅m⋅m ⋅ℯ                                      
H_{13}__{int} = - ───────────────────────────────────────────────────────── - 
                                      -α + δ_j2 - 2⋅δₖ₁                       

                               †                                              
          __  †    †  - ⅈ⋅α⋅t⋅m ⋅m + ⅈ⋅δ_j1⋅t - 2⋅ⅈ⋅δₖ₁⋅t          __ __    † 
2⋅α⋅λ₁⋅ξ₁⋅λ₁⋅m ⋅m⋅m ⋅ℯ                                      2⋅α⋅λ₁⋅λ₁⋅ξ₁⋅m⋅m ⋅
───────────────────────────────────────────────────────── - ──────────────────
                    -α + δ_j1 - 2⋅δₖ₁                                         

            †                                                        †        
   ⅈ⋅α⋅t⋅m⋅m  - ⅈ⋅δ_j1⋅t + 2⋅ⅈ⋅δₖ₁⋅t          __ __    †    ⅈ⋅α⋅t⋅m⋅m  - ⅈ⋅δ_j
m⋅ℯ                                    2⋅α⋅λ₂⋅λ₁⋅ξ

Now we've got our building blocks ready. Please note, however, that when integrating, we swept the qubit operators that should still be appearing in the denominators under the rug. Otherwise, we ran into problems in the next step. But after all, operators in the denominator are problematic anyway... 

So, brace yourself for the multiplication - the expression will be rather lengthy!

In [34]:
H_mul = ((H10*H13_int).expand()); Eq(symbols("H_{mul}"), H_mul)

                                         †                                †   
             2      3 __ __  †  - ⅈ⋅α⋅t⋅m ⋅m - ⅈ⋅δ_j1⋅t  †    †  - ⅈ⋅α⋅t⋅m ⋅m 
          2⋅α ⋅λ₁⋅ξ₁ ⋅λ₂⋅ξ₁⋅m ⋅ℯ                       ⋅m ⋅m⋅m ⋅ℯ             
H_{mul} = ────────────────────────────────────────────────────────────────────
                                              -α + δ_j2 - 2⋅δₖ₁               

                                                        †                     
+ ⅈ⋅δ_j2⋅t - 2⋅ⅈ⋅δₖ₁⋅t      2      3 __ __  †  - ⅈ⋅α⋅t⋅m ⋅m - 2⋅ⅈ⋅δ_j1⋅t + ⅈ⋅δ
                         2⋅α ⋅λ₁⋅ξ₁ ⋅λ₂⋅ξ₂⋅m ⋅ℯ                               
────────────────────── + ─────────────────────────────────────────────────────
                                                                    -α + δ_j2 

                        †                                                     
_j2⋅t  †    †  - ⅈ⋅α⋅t⋅m ⋅m + ⅈ⋅δ_j2⋅t - 2⋅ⅈ⋅δₖ₁⋅t      2      3 __ __  †  - ⅈ
     ⋅m ⋅m⋅m ⋅ℯ                                   

Fortunately, the partial trace over the qubit ground state is up next, meaning we can get ride of quite a few terms.

In [36]:
H_mul_trace = trace(H_mul); Eq(symbols("H_{mul}^{traced}"),H_mul_trace)

                                                                              
                       2      3  ⅈ⋅t⋅(-2⋅α - δ_j1 + δ_j2 - 2⋅δₖ₁) __ __  †    
                    2⋅α ⋅λ₁⋅ξ₁ ⋅ℯ                                ⋅λ₂⋅ξ₁⋅m ⋅m  
H_{mul}__{traced} = ──────────────────────────────────────────────────────── +
                                          δ_j2 - 2⋅δₖ₁                        

                                                                              
    2      3  2⋅ⅈ⋅t⋅(-α - δ_j1 + δ_j2 - δₖ₁) __ __  †        2      3  ⅈ⋅t⋅(-2
 2⋅α ⋅λ₁⋅ξ₁ ⋅ℯ                              ⋅λ₂⋅ξ₂⋅m ⋅m   2⋅α ⋅λ₁⋅ξ₁ ⋅ℯ       
 ────────────────────────────────────────────────────── + ────────────────────
                      δ_j2 - 2⋅δₖ₁                                            

                                                                              
⋅α - δ_j1 + δ_j2 - 2⋅δₖ₁) __ __  †        2      3  -2⋅ⅈ⋅t⋅(α + δₖ₁) __ __  † 
                         ⋅λ₁⋅ξ₂⋅m ⋅m   2⋅α ⋅λ₁⋅ξ₁ 

Nearly there, we can now apply a resonance condition and perform time averaging. Which coefficients are we left with?

In [37]:
time_average(H_mul_trace, [-dj1 + dj2 - 2*dk1])

   2      3 __ __  †        2      3 __ __  †  
2⋅α ⋅λ₁⋅ξ₁ ⋅λ₂⋅ξ₁⋅m ⋅m   2⋅α ⋅λ₁⋅ξ₁ ⋅λ₁⋅ξ₂⋅m ⋅m
────────────────────── + ──────────────────────
     δ_j2 - 2⋅δₖ₁             δ_j1 - 2⋅δₖ₁     

As we can see, these coefficients indeed correspond to half of those in equation (3.17) in Xinyu's thesis. Success! 
We're already at the end of this little tutorial... next steps await! Here's a little to-do list:
- Get all transformations to work.
- Test $H_{10}$ and $H_{13}$ with more terms from the generating for-loops. Can we get the remaining terms from (3.17)?
- Test further terms in second-order RWA.
- Test higher-order RWA.

Have fun with Sympsi! :D 